<a href="https://colab.research.google.com/github/fnunezs/geopandas/blob/main/convexhull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualización de envolvente convexa (convex hull) mediante Python

El siguiente libro de notas genera un mapa con una serie de envolventes convexas generadas a partir de las localizaciones geográficas de accidentes de tráfico (atropellos graves) ocurridos en la ciudad de Madrid durante mayo de 2021.

Fuente de datos:    
Portal de Datos Abiertos del Ayuntamiento de Madrid:
 [Accidentes de tráfico en la ciudad de Madrid](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=7c2843010d9c3610VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default)

Instalamos las librerías necesarias:

In [1]:
!pip install pandas
!pip install geopandas
!pip install pyshp
!pip install shapely
!pip install descartes
!pip install mapclassify
!pip install --upgrade folium 
!pip install fiona
!pip install utm

%matplotlib inline
import fiona
import folium
import mapclassify
import pandas as pd
import geopandas
import utm
from geopandas import GeoSeries
from shapely.geometry import Polygon, Point
import matplotlib.pyplot as plt

Montamos la unidad que contiene el conjunto de datos extraído del Portal de Datos Abiertos del Ayuntamiento de Madrid:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Cargamos el conjunto de datos que contiene los atropellos graves producidos en la ciudad de Madrid en mayo de 2021:

In [3]:
df_accidentes = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/202105_accidentes_graves3.csv", sep=";")

Definimos una función para convertir las coordenadas UTM de los lugares de los accidentes al sistema EPSG:4326

In [4]:
def utm_to_latlon(coords, zone_number, zone_letter):
  easting = coords[0]
  northing = coords[1]
  return list(reversed(utm.to_latlon(easting, northing, zone_number, zone_letter)))

Convertimos las coordenadas de los lugares de los accidentes:

In [5]:
df_accidentes["lat_lon"] = list(zip(df_accidentes["coordenada_x_utm"], df_accidentes["coordenada_y_utm"]))
df_accidentes["lat_lon"] = [utm_to_latlon(xy, 30, "N") for xy in df_accidentes.lat_lon]

geometry = [Point(xy) for xy in df_accidentes["lat_lon"]]
gdf_accidentes = geopandas.GeoDataFrame(df_accidentes, crs="EPSG:4326", geometry=geometry)

gdf_accidentes.set_crs(epsg=4326, inplace=True)

,num_expediente,fecha,hora,localizacion,numero,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,lesividad,coordenada_x_utm,coordenada_y_utm,lat_lon,geometry
0,2021S007368,01/05/2021,17:40:00,"CALL. FUENCARRAL, 56",56,CENTRO,Atropello a persona,Despejado,Turismo,Conductor,De 60 a 64 años,Hombre,7.0,440570.53,4475032.91,"[-3.7005714798145273, 40.423806368762754]",POINT (-3.70057 40.42381)
1,2021S007368,01/05/2021,17:40:00,"CALL. FUENCARRAL, 56",56,CENTRO,Atropello a persona,Despejado,Turismo,Peatón,De 45 a 49 años,Mujer,7.0,440570.53,4475032.91,"[-3.7005714798145273, 40.423806368762754]",POINT (-3.70057 40.42381)
2,2021S007369,01/05/2021,17:50:00,"CALL. BLASCO DE GARAY, 62",62,CHAMBERÍ,Atropello a persona,Despejado,Furgoneta,Conductor,De 45 a 49 años,Hombre,14.0,439673.54,4476373.31,"[-3.7112718923750063, 40.435816722952076]",POINT (-3.71127 40.43582)
3,2021S007369,01/05/2021,17:50:00,"CALL. BLASCO DE GARAY, 62",62,CHAMBERÍ,Atropello a persona,Despejado,Furgoneta,Peatón,Menor de 5 años,Hombre,7.0,439673.54,4476373.31,"[-3.7112718923750063, 40.435816722952076]",POINT (-3.71127 40.43582)
4,2021S007369,01/05/2021,17:50:00,"CALL. BLASCO DE GARAY, 62",62,CHAMBERÍ,Atropello a persona,Despejado,Turismo,Conductor,Más de 74 años,Hombre,14.0,439673.54,4476373.31,"[-3.7112718923750063, 40.435816722952076]",POINT (-3.71127 40.43582)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,2021S009509,31/05/2021,14:25:00,"CALL. SAN MATEO, 8",8,CENTRO,Atropello a persona,Despejado,Turismo,Peatón,Menor de 5 años,Hombre,1.0,440624.96,4475187.88,"[-3.6999443601105497, 40.42520629685752]",POINT (-3.69994 40.42521)
228,2021S009584,29/05/2021,15:40:00,"AVDA. RAFAELA YBARRA, 20",20,USERA,Atropello a persona,NaN,Turismo,Conductor,De 18 a 20 años,Mujer,NaN,439616.82,4470388.63,"[-3.7113726985892472, 40.38189980781438]",POINT (-3.71137 40.38190)
229,2021S009584,29/05/2021,15:40:00,"AVDA. RAFAELA YBARRA, 20",20,USERA,Atropello a persona,NaN,Turismo,Peatón,De 40 a 44 años,Hombre,NaN,439616.82,4470388.63,"[-3.7113726985892472, 40.38189980781438]",POINT (-3.71137 40.38190)
230,2021S009729,31/05/2021,8:46:00,AVDA. INGENIERO EMILIO HERRERA / CALL. INFANTA...,5,HORTALEZA,Atropello a persona,Despejado,Turismo,Conductor,De 25 a 29 años,Mujer,14.0,444549.26,4482429.27,"[-3.6543181721285936, 40.49071116391524]",POINT (-3.65432 40.49071)


Creamos un mapa centrado en la ciudad de Madrid y representamos los lugares de los accidentes, generando la envolvente convexa de estos puntos geográficos:

In [6]:
# Creamos un mapa centrado en la ciudad de Madrid
mapa_accidentes = folium.Map(location=[40.429606584571246, -3.705951286477693], zoom_start=12, tiles='CartoDB positron')

# Añadimos al mapa los puntos donde se han producido accidentes
for _, r in gdf_accidentes.iterrows():   
  color = lambda x : "red" if (x > 2 and x < 5) else "orange"
  sim_geo = geopandas.GeoSeries(r['geometry'])
  geo_j = sim_geo.to_json()
  geo_j = folium.GeoJson(data=geo_j,
                         style_function=lambda x: {'fillColor': 'red'}, 
                         marker = folium.Marker(icon=folium.Icon(color=color(r['lesividad']), icon="car", prefix='fa'), size = (10,10)))
  descripcion = "Vehículo implicado: " + str(r['tipo_vehiculo']) + "\n" + "Edad victima: " + str(r['rango_edad']) 

  folium.Popup(descripcion).add_to(geo_j)
  geo_j.add_to(mapa_accidentes)

# Generamos convex-hull a partir de los puntos de los accidentes
sim_geo = gdf_accidentes['geometry'].unary_union.convex_hull

# Añadimos el polígono generado al mapa
convex_hull_accidentes = geopandas.GeoSeries([sim_geo]).to_json()
convex_hull_accidentes = folium.GeoJson(data=convex_hull_accidentes, 
                                 style_function=lambda x: {'fillColor': 'orange', 'color': 'white'})

convex_hull_accidentes.add_to(mapa_accidentes)

region_accidentes_graves = gdf_accidentes[gdf_accidentes.lesividad.between(3,4)]
# Generamos convex-hull a partir de los puntos de los accidentes
sim_geo_graves = region_accidentes_graves['geometry'].unary_union.convex_hull

# Añadimos el polígono generado al mapa
convex_hull_accidentes_graves = geopandas.GeoSeries([sim_geo_graves]).to_json()
convex_hull_accidentes_graves = folium.GeoJson(data=convex_hull_accidentes_graves, 
                                 style_function=lambda x: {'fillColor': 'red', 'color': 'black'})

mapa_accidentes


A continuación vamos a cargar otro conjunto de datos que contiene la localización de los lugares en los que los agentes de movilidad del Ayuntamiento de Madrid multaron a conductores que no habían respetado los semáforos durante mayo de 2021: 

In [7]:
df_multas = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/202105_multas_trafico.csv", sep=";")
df_multas.head()


,CALIFICACION,LUGAR,MES,ANIO,HORA,IMP_BOL,DESCUENTO,PUNTOS,DENUNCIANTE,HECHO-BOL,COORDENADA-X,COORDENADA-Y
0,GRAVE,123 GNRAL RICARDOS - PSO QUINC,5,2021,1.59,200.0,SI,4,AGENTES DE MOVILIDAD,REBASAR UN SEMÁFORO EN FASE ROJA. ...,439232.86,4472262.78
1,GRAVE,117 JOSE ABASCAL - SANTA ENGRA,5,2021,2.06,200.0,SI,4,AGENTES DE MOVILIDAD,REBASAR UN SEMÁFORO EN FASE ROJA. ...,440671.00,4476875.00
2,GRAVE,098 AV MEDITERRANEO 32,5,2021,6.33,200.0,SI,4,AGENTES DE MOVILIDAD,REBASAR UN SEMÁFORO EN FASE ROJA. ...,443017.57,4473349.72
3,GRAVE,110 AV MARQUES DE CORBERA - RI,5,2021,7.21,200.0,SI,4,AGENTES DE MOVILIDAD,REBASAR UN SEMÁFORO EN FASE ROJA. ...,444461.78,4475104.19
4,GRAVE,121 CRA. CARABANCHEL ARAVACA -,5,2021,7.31,200.0,SI,4,AGENTES DE MOVILIDAD,REBASAR UN SEMÁFORO EN FASE ROJA. ...,434397.30,4472312.57


Convertimos las coordenadas UTM de los lugares en los que se han producido las sanciones al sistema EPSG:4326

In [8]:
df_multas["lat_lon"] = list(zip(df_multas["COORDENADA-X"], df_multas["COORDENADA-Y "]))
df_multas["lat_lon"] = [utm_to_latlon(xy, 30, "N") for xy in df_multas.lat_lon]

geometry = [Point(xy) for xy in df_multas["lat_lon"]]
gdf_multas = geopandas.GeoDataFrame(df_multas, crs="EPSG:4326", geometry=geometry)

gdf_multas.set_crs(epsg=4326, inplace=True)

,CALIFICACION,LUGAR,MES,ANIO,HORA,IMP_BOL,DESCUENTO,PUNTOS,DENUNCIANTE,HECHO-BOL,COORDENADA-X,COORDENADA-Y,lat_lon,geometry
0,GRAVE,123 GNRAL RICARDOS - PSO QUINC,5,2021,1.59,200.0,SI,4,AGENTES DE MOVILIDAD,REBASAR UN SEMÁFORO EN FASE ROJA. ...,439232.86,4472262.78,"[-3.7160745932618626, 40.39875515358146]",POINT (-3.71607 40.39876)
1,GRAVE,117 JOSE ABASCAL - SANTA ENGRA,5,2021,2.06,200.0,SI,4,AGENTES DE MOVILIDAD,REBASAR UN SEMÁFORO EN FASE ROJA. ...,440671.00,4476875.00,"[-3.699559142022515, 40.44040792470855]",POINT (-3.69956 40.44041)
2,GRAVE,098 AV MEDITERRANEO 32,5,2021,6.33,200.0,SI,4,AGENTES DE MOVILIDAD,REBASAR UN SEMÁFORO EN FASE ROJA. ...,443017.57,4473349.72,"[-3.671576215219203, 40.408814486970535]",POINT (-3.67158 40.40881)
3,GRAVE,110 AV MARQUES DE CORBERA - RI,5,2021,7.21,200.0,SI,4,AGENTES DE MOVILIDAD,REBASAR UN SEMÁFORO EN FASE ROJA. ...,444461.78,4475104.19,"[-3.654709485043287, 40.424717345569306]",POINT (-3.65471 40.42472)
4,GRAVE,121 CRA. CARABANCHEL ARAVACA -,5,2021,7.31,200.0,SI,4,AGENTES DE MOVILIDAD,REBASAR UN SEMÁFORO EN FASE ROJA. ...,434397.30,4472312.57,"[-3.7730567975709213, 40.398836785023335]",POINT (-3.77306 40.39884)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4241,GRAVE,099 MENENDEZ PELAYO H. NIÑO JE,5,2021,21.38,200.0,SI,4,AGENTES DE MOVILIDAD,REBASAR UN SEMÁFORO EN FASE ROJA. ...,442633.05,4474191.88,"[-3.6761836798737875, 40.41637470099567]",POINT (-3.67618 40.41637)
4242,GRAVE,123 GNRAL RICARDOS - PSO QUINC,5,2021,22.00,200.0,SI,4,AGENTES DE MOVILIDAD,REBASAR UN SEMÁFORO EN FASE ROJA. ...,439232.86,4472262.78,"[-3.7160745932618626, 40.39875515358146]",POINT (-3.71607 40.39876)
4243,GRAVE,123 GNRAL RICARDOS - PSO QUINC,5,2021,22.08,200.0,SI,4,AGENTES DE MOVILIDAD,REBASAR UN SEMÁFORO EN FASE ROJA. ...,439232.86,4472262.78,"[-3.7160745932618626, 40.39875515358146]",POINT (-3.71607 40.39876)
4244,GRAVE,114 ALBERTO AGUILERA - BALTASA,5,2021,22.28,200.0,SI,4,AGENTES DE MOVILIDAD,REBASAR UN SEMÁFORO EN FASE ROJA. ...,439802.04,4475938.23,"[-3.7097157285708477, 40.431906645079216]",POINT (-3.70972 40.43191)


Generamos un nuevo mapa para visualizar la envolvente convexa de los lugares en los que se han producido sanciones (azul) junto a las envolventes de los atropellos graves (naranja) y muy graves (rojo) cargados anteriormente:

In [9]:
mapa_multas_accidentes = folium.Map(location=[40.429606584571246, -3.705951286477693], zoom_start=12, tiles='CartoDB positron')

# Generamos convex-hull a partir de los puntos de las multas
sim_geo = gdf_multas['geometry'].unary_union.convex_hull

convex_hull_multas = geopandas.GeoSeries([sim_geo]).to_json()
convex_hull_multas = folium.GeoJson(data=convex_hull_multas, 
                                 style_function=lambda x: {'fillColor': 'blue', 'color': 'blue'})


# Añadimos los polígonos al mapa
convex_hull_multas.add_to(mapa_multas_accidentes)
convex_hull_accidentes.add_to(mapa_multas_accidentes)
convex_hull_accidentes_graves.add_to(mapa_multas_accidentes)

mapa_multas_accidentes